## Catalog Geophone 3 and Geophone 4 events

### Import libraries

In [1]:
import pandas as pd
from obspy import read,UTCDateTime
from datetime import datetime, timedelta
import numpy as np
import os
import glob
import sys
import matplotlib.pyplot as plt
from PIL import Image

# Import functions
fxndir = '../functions/'
sys.path.insert(0,fxndir)
from moon2data import *

### Load catalogs of all high quality moonquakes

In [2]:
# Catalog with information by geophone
mqdir = '../catalogs/final_catalogs/'
cat1 = pd.read_csv(mqdir + 'A17_moonquakes_catalog_HQ_final.csv')
cat1.drop(list(cat1.filter(regex='Unnamed|index')), axis=1, inplace=True)

# Catalog with information averaged per event
cat2 = pd.read_csv(mqdir + 'A17_moonquakes_catalog_HQ_avg_event_stats.csv')
cat2.drop(list(cat2.filter(regex='Unnamed|index')), axis=1, inplace=True)

### Inputs to obtain waveforms

In [3]:
parentdir = '/data/ytamama/Apollo17/LSPE_data/sac_volts_ds/'
minfreq = 3
maxfreq = 35
befwin = 10
aftwin = 20

### Geophone 3 Events

#### From visual inspection of candidate events, we identified the Geophone 3 families

In [4]:
evid_refs = ['761028-01-M1','761028-23-M3','761030-08-M1','761231-15-M1','770310-23-M4', '770225-21-M1']
cat1_geo3 = cat1[cat1['evid_ref'].isin(evid_refs)]
evids_geo3_rpt = np.unique(cat1_geo3.evid.tolist())
len(np.unique(cat1_geo3.evid_ref.tolist()))

6

#### ... and the isolated Geophone 3 events 

In [5]:
evids_isolated = ['760815-21-M1','760831-14-M4','760902-07-M2','760909-03-N1','760912-02-N2','760916-04-N1','760930-21-M2',
                 '761004-01-M1','761011-12-M1','761029-17-M2','761102-12-M2','761110-03-M1','761112-06-M1','761127-10-M3',
                 '761128-06-M4','761129-19-M2','761227-21-M1','770108-07-M4','770110-06-M2','770126-00-N1','770126-11-M2',
                 '770128-00-M1','770208-10-M2','770309-22-M3','770325-20-M1','770325-20-M2','770327-11-M3','760930-02-N2',
                 '770405-06-M4','770406-18-M5','770408-12-M5','770423-21-M6','770206-22-M4','760815-17-N2','770205-16-M4',
                 '761001-18-M1','770215-02-N1','770223-19-N2','770226-17-M1','761010-12-N2','761011-07-N3','761031-06-M2',
                 '761109-05-N1','770301-02-M1','770309-08-M2','761128-06-M4','761128-19-M3','761129-19-M2','761208-23-N3',
                 '770310-21-N1','770409-03-M2','761212-00-M2','761227-21-M1','761228-15-M1','770108-07-M4','770110-06-M2',
                 '770126-11-M2']
evids_isolated = np.unique(evids_isolated)
evids_geo3 = np.concatenate([evids_geo3_rpt,evids_isolated])

#### Plot waveforms of selected Geophone 3 events as a sanity check

In [6]:
# Substitute with your own directory
savedir = '/data/ytamama/figures/Apollo17/geo3_events/' 
pngnames = []

# Iteratively plot events
for evid in evids_geo3:
    row = cat2.loc[cat2.evid == evid].iloc[0]
    evid_ref = row.evid_ref
    arrtime = datetime.strptime(row.avg_picktime_SNR, '%Y-%m-%d %H:%M:%S.%f')
    st1 = moon2sac(arrtime,1,befwin,aftwin,minfreq,maxfreq,parentdir)
    trdata1 = st1.traces[0].data
    trtimes1 = st1.traces[0].times() - befwin
    st2 = moon2sac(arrtime,2,befwin,aftwin,minfreq,maxfreq,parentdir)
    trdata2 = st2.traces[0].data
    trtimes2 = st2.traces[0].times() - befwin
    st3 = moon2sac(arrtime,3,befwin,aftwin,minfreq,maxfreq,parentdir)
    trdata3 = st3.traces[0].data
    trtimes3 = st3.traces[0].times() - befwin
    st4 = moon2sac(arrtime,4,befwin,aftwin,minfreq,maxfreq,parentdir)
    trdata4 = st4.traces[0].data
    trtimes4 = st4.traces[0].times() - befwin

    # Normalize
    trdata1_norm = trdata1 / np.max(np.abs(trdata1))
    trdata2_norm = trdata2 / np.max(np.abs(trdata1))
    trdata3_norm = trdata3 / np.max(np.abs(trdata1))
    trdata4_norm = trdata4 / np.max(np.abs(trdata1))

    # Initialize figure
    fig,ax = plt.subplots(1,1,figsize=(8, 5))
    ax.plot(trtimes1,trdata1_norm+6, color='C0')
    ax.plot(trtimes2,trdata2_norm+4, color='C1')
    ax.plot(trtimes3,trdata3_norm+2, color='C2')
    ax.plot(trtimes4,trdata4_norm, color='C3')
    ax.set_title(f'EVID {evid}, (REF: {evid_ref})',fontweight='bold')
    ax.set_xlim([-1*befwin,aftwin])

    # Save figure
    plt.savefig(savedir + 'EVID' + evid + '_waveforms.png', bbox_inches="tight")
    pngnames.append(savedir + 'EVID' + evid + '_waveforms.png')
    plt.close()

In [7]:
# Combine figures into one PDF
images = [
    Image.open(f)
    for f in pngnames
]
geo34_dir = './geo3_geo4_waveforms/'
pdf_path = geo34_dir + 'geo3_events.pdf'
images[0].save(
    pdf_path, "PDF" ,resolution=100.0, save_all=True, append_images=images[1:]
)

for pngname in pngnames:
    os.system('rm ' + pngname)

#### Isolate Geophone 3 events from catalogs

In [8]:
# Catalog 1
cat1_geo3 = cat1[cat1['evid'].isin(evids_geo3)]
cat1_geo3 = cat1_geo3.sort_values(by=['interval_day_number'],ignore_index=True)
cat1_geo3 = cat1_geo3.reset_index()
cat1_geo3 = cat1_geo3.drop(columns=['index'])

# Catalog 2
cat2_geo3 = cat2[cat2['evid'].isin(evids_geo3)]
cat2_geo3 = cat2_geo3.sort_values(by=['avg_interval_day'],ignore_index=True)
cat2_geo3 = cat2_geo3.reset_index()
cat2_geo3 = cat2_geo3.drop(columns=['index'])

#### Save catalogs 

In [9]:
# Geophone 3 events
mqdir2 = mqdir + 'geo3_geo4_events/'
cat1_geo3.to_csv(mqdir2 + 'Geophone3_events_catalog_HQ_final.csv',index=False)
cat2_geo3.to_csv(mqdir2 + 'Geophone3_events_catalog_HQ_avg_event_stats.csv',index=False)

### Geophone 4 Events

#### From visual inspection, identify families with an impulsive signal arriving first at Geophone 4

In [10]:
evid_refs = ['770206-17-M2','770406-21-N2']
cat1_geo4 = cat1[cat1['evid_ref'].isin(evid_refs)]
evids_geo4_rpt = np.unique(cat1_geo4.evid.tolist())

#### Identify isolated events also with this trait

In [11]:
evids_isolated = ['760911-23-M4','760913-12-M2','761111-14-M5','761211-05-M2','770208-07-N1','770408-13-M2',
                 '760831-17-M5','761111-13-M4','761111-13-N2','761209-12-N1','770421-00-M2','770408-11-M7',
                 '760913-12-M6']
evids_geo4 = np.concatenate([evids_geo4_rpt,evids_isolated])

#### Plot waveforms of Geophone 4 events as a sanity check

In [12]:
# Substitute with your own directory
savedir = '/data/ytamama/figures/Apollo17/geo4_events/'
pngnames = []

# Iteratively plot events
for evid in evids_geo4:
    row = cat2.loc[cat2.evid == evid].iloc[0]
    evid_ref = row.evid_ref
    arrtime = datetime.strptime(row.avg_picktime_SNR, '%Y-%m-%d %H:%M:%S.%f')
    st1 = moon2sac(arrtime,1,befwin,aftwin,minfreq,maxfreq,parentdir)
    trdata1 = st1.traces[0].data
    trtimes1 = st1.traces[0].times() - befwin
    st2 = moon2sac(arrtime,2,befwin,aftwin,minfreq,maxfreq,parentdir)
    trdata2 = st2.traces[0].data
    trtimes2 = st2.traces[0].times() - befwin
    st3 = moon2sac(arrtime,3,befwin,aftwin,minfreq,maxfreq,parentdir)
    trdata3 = st3.traces[0].data
    trtimes3 = st3.traces[0].times() - befwin
    st4 = moon2sac(arrtime,4,befwin,aftwin,minfreq,maxfreq,parentdir)
    trdata4 = st4.traces[0].data
    trtimes4 = st4.traces[0].times() - befwin

    # Normalize
    trdata1_norm = trdata1 / np.max(np.abs(trdata1))
    trdata2_norm = trdata2 / np.max(np.abs(trdata1))
    trdata3_norm = trdata3 / np.max(np.abs(trdata1))
    trdata4_norm = trdata4 / np.max(np.abs(trdata1))

    # Initialize figure
    fig,ax = plt.subplots(1,1,figsize=(8, 5))
    ax.plot(trtimes1,trdata1_norm+6, color='C0')
    ax.plot(trtimes2,trdata2_norm+4, color='C1')
    ax.plot(trtimes3,trdata3_norm+2, color='C2')
    ax.plot(trtimes4,trdata4_norm, color='C3')
    ax.set_title(f'EVID {evid}, (REF: {evid_ref})',fontweight='bold')
    ax.set_xlim([-1*befwin,aftwin])

    # Save figure
    plt.savefig(savedir + 'EVID' + evid + '_waveforms.png', bbox_inches="tight")
    pngnames.append(savedir + 'EVID' + evid + '_waveforms.png')
    plt.close()

In [13]:
# Combine figures into one PDF
images = [
    Image.open(f)
    for f in pngnames
]
pdf_path = geo34_dir + 'geo4_events.pdf'
images[0].save(
    pdf_path, "PDF" ,resolution=100.0, save_all=True, append_images=images[1:]
)

for pngname in pngnames:
    os.system('rm ' + pngname)

#### Isolate Geophone 4 events from catalogs

In [14]:
# Catalog 1
cat1_geo4 = cat1[cat1['evid'].isin(evids_geo4)]
cat1_geo4 = cat1_geo4.sort_values(by=['interval_day_number'],ignore_index=True)
cat1_geo4 = cat1_geo4.reset_index()
cat1_geo4 = cat1_geo4.drop(columns=['index'])

# Catalog 2
cat2_geo4 = cat2[cat2['evid'].isin(evids_geo4)]
cat2_geo4 = cat2_geo4.sort_values(by=['avg_interval_day'],ignore_index=True)
cat2_geo4 = cat2_geo4.reset_index()
cat2_geo4 = cat2_geo4.drop(columns=['index'])

#### Save catalogs 

In [15]:
# Geophone 4 events
cat1_geo4.to_csv(mqdir2 + 'Geophone4_events_catalog_HQ_final.csv',index=False)
cat2_geo4.to_csv(mqdir2 + 'Geophone4_events_catalog_HQ_avg_event_stats.csv',index=False)